# UberEats - Reconise Text using CRNN pre-built pytorch model

Download the pre built model and place in models folder 

In [1]:
#!wget https://www.dropbox.com/s/l0vo83hmvv2aipn/crnn.pth

### Imports

In [9]:
import crnn.utils as utils 
import crnn.dataset as dataset
import torch
import string
from torch.autograd import Variable
from PIL import Image
import crnn.models.crnn as crnn
import glob
from IPython.core import display

In [10]:
model_path = '/home/micdsouz/wrkdir/ctpn/src/UberMenu/models/crnn.pth'
# Global Variables
alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'
print("Character List: ", alphabet)

Character List:  0123456789abcdefghijklmnopqrstuvwxyz


In [11]:
model = crnn.CRNN(32, 1, 37, 256, 1)
print('loading pretrained model from %s' % model_path)

loading pretrained model from /home/micdsouz/wrkdir/ctpn/src/UberMenu/models/crnn.pth


In [12]:
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [13]:
converter = utils.strLabelConverter(alphabet)
transformer = dataset.resizeNormalize((100, 32))

In [14]:
def extract_text(img_path):
    image = Image.open(img_path).convert('L')
    image = transformer(image)
    image = image.view(1, *image.size())
    image = Variable(image)
    model.eval()
    preds = model(image)
    _, preds = preds.max(2)
    preds = preds.transpose(1, 0).contiguous().view(-1)
    preds_size = Variable(torch.IntTensor([preds.size(0)]))
    raw_pred = converter.decode(preds.data, preds_size.data, raw=True)
    sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
    display.display(display.Image(img_path))
    print('%-20s => %-20s' % (raw_pred, sim_pred))
    return raw_pred, sim_pred

In [15]:
extract_text('/home/micdsouz/wrkdir/ctpn/src/UberMenu/data/test/1.png')

b----a--s-t-o--n-c-i-t-o-- => bastoncito          


('b----a--s-t-o--n-c-i-t-o--', 'bastoncito')